In [29]:
import tracemalloc
import time
import scanpy as sc
import pandas as pd

import cupy as cp
import cupyx
import numpy as np
from tqdm import tqdm

from icecream import ic

from SEACells.core import SEACells


In [30]:
from scipy.sparse import csr_matrix

In [31]:
ad = sc.read("/home/aparna/DATA/aparnakumar/50000_cells/mouse_marioni_50k.h5ad") 
num_cells = 1000
ad = ad[:num_cells]  

In [32]:
## Core parameters 
# number of SEACells
n_SEACells = num_cells // 75
build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                            # This would be replaced by 'X_svd' for ATAC data

## Additional parameters
n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells

In [33]:
model2 = SEACells(ad, 
                 use_gpu=False, 
                 use_sparse=True, 
                 build_kernel_on=build_kernel_on, 
                 n_SEACells=n_SEACells, 
                 n_waypoint_eigs=n_waypoint_eigs,
                 convergence_epsilon = 1e-5)

model2.construct_kernel_matrix()
model2.initialize_archetypes()
model2.initialize()

SPARSE AND NOT GPU
TRYING SEACellsCPU
Welcome to SEACells!
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...



Constructing CSR matrix...
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...


100%|██████████| 14/14 [00:00<00:00, 713.80it/s]

Done.
Sampling waypoints ...
Done.
Selecting 9 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...
Selecting 4 cells from greedy initialization.
Randomly initialized A matrix.


Setting convergence threshold at 0.00056


In [34]:
A = model2.A_

B = model2.B_

K = model2.K

In [35]:
ic(type(A))
ic(type(B))
ic(type(K))

ic| type(A): <class 'scipy.sparse.csc.csc_matrix'>
ic| type(B): <class 'scipy.sparse.csr.csr_matrix'>
ic| type(K): <class 'scipy.sparse.csr.csr_matrix'>


scipy.sparse.csr.csr_matrix

In [36]:
dict_A = {'cpu': A, 'gpu_dense': cp.array(A.todense()), 'gpu_sparse': cp.sparse.csr_matrix(A)}
dict_B = {'cpu': B, 'gpu_dense': cp.array(B.todense()), 'gpu_sparse': cp.sparse.csr_matrix(B)}
dict_K = {'cpu': K, 'gpu_dense': cp.array(K.todense()), 'gpu_sparse': cp.sparse.csr_matrix(K)}


In [54]:
# Print types of everything in the dictionary
for key, value in dict_A.items():
    ic(key, type(value))

for key, value in dict_B.items():
    ic(key, type(value))

for key, value in dict_K.items():
    ic(key, type(value))
    

ic| key: 'cpu', type(value): <class 'scipy.sparse.csc.csc_matrix'>
ic| key: 'gpu_dense', type(value): <class 'cupy.ndarray'>


ic| key: 'gpu_sparse'
    type(value): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| key: 'cpu', type(value): <class 'scipy.sparse.csr.csr_matrix'>
ic| key: 'gpu_dense', type(value): <class 'cupy.ndarray'>
ic| key: 'gpu_sparse'
    type(value): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| key: 'cpu', type(value): <class 'scipy.sparse.csr.csr_matrix'>
ic| key: 'gpu_dense', type(value): <class 'cupy.ndarray'>
ic| key: 'gpu_sparse'
    type(value): <class 'cupyx.scipy.sparse._csr.csr_matrix'>


In [90]:
def compute_sparsity(M):
    """
    Compute the proportion of non-zero elements in a matrix
    @param M: a dense numpy array
    """
    return (M != 0).sum() / M.size

def check_results(d):
    """
    Check that all results in a dictionary are the same
    """
    
    v2 = d['cpu']
    # Turn v2 into a numpy array
    v2 = v2.toarray()
    elt2 = np.where(v2.flatten() != 0)[0]
    print('Sparsity of {} is {}'.format('CPU', compute_sparsity(v2)))
    for key in ['gpu_sparse', 'gpu_dense']:
        v1 = d[key]
        # Convert to dense for comparison
        if key == 'gpu_sparse':
            v1 = cp.array(v1.todense())
        # Turn into a numpy array
        v1 = cp.asnumpy(v1)
        print('Sparsity of {} is {}'.format(key, compute_sparsity(v1)))
        # Grab the nonzero elements of v1
        elt1 = np.where(v1.flatten() != 0)[0]
        # Check that the nonzero elements are the same
        try:
            assert set(list(elt1)) == set(list(elt2))
        except AssertionError:
            print('Nonzero elements of {} are different from nonzero elements of {}'.format('CPU', key))
            


# Check that all the matrices are the same 
for d in [dict_A, dict_B, dict_K]:
    check_results(d)

Sparsity of CPU is 0.9603846153846154
Sparsity of gpu_sparse is 0.9603846153846154
Sparsity of gpu_dense is 0.9603846153846154
Sparsity of CPU is 0.001
Sparsity of gpu_sparse is 0.001
Sparsity of gpu_dense is 0.001
Sparsity of CPU is 0.095394
Sparsity of gpu_sparse is 0.095394
Sparsity of gpu_dense is 0.095394


In [91]:
t2 = {}
t1 = {}

for key in ['cpu', 'gpu_dense', 'gpu_sparse']: 
    A = dict_A[key] 
    B = dict_B[key] 
    K = dict_K[key] 

    t2[key] = (K @ B).T
    t1[key] = t2[key] @ B

check_results(t1)
check_results(t2)

Sparsity of CPU is 0.1834319526627219
Sparsity of gpu_sparse is 0.1834319526627219
Sparsity of gpu_dense is 0.1834319526627219
Sparsity of CPU is 0.09507692307692307
Sparsity of gpu_sparse is 0.09507692307692307
Sparsity of gpu_dense is 0.09507692307692307


In [93]:
G ={}

for key in ['cpu', 'gpu_dense', 'gpu_sparse']: 
    A = dict_A[key] 
    B = dict_B[key] 
    K = dict_K[key] 

    temp = 2.0 * (t1[key] @ A - t2[key])

    # Make everything less than 1e-15 equal to 0 
    print("key: ", key)
    G[key] = temp
    # # Type of temp can be numpy sparse matrix, cupy dense matrix, or cupy sparse matrix
    # try:
    #     temp[temp < 1e-15] = 0
    #     G[key] = temp
    # except Exception as e:
    #     try: 
    #         temp.data[temp.data < 1e-15] = 0
    #         G[key] = temp
    #     except Exception as e:
    #         print(e)

check_results(G)

key:  cpu
key:  gpu_dense
key:  gpu_sparse
Sparsity of CPU is 0.9856153846153846
Sparsity of gpu_sparse is 0.9856153846153846
Sparsity of gpu_dense is 0.9856153846153846


In [104]:
# GPU Dense + Sparse

egs = {} 

for key in ['cpu', 'gpu_dense', 'gpu_sparse']:
    Gg = G[key]

    Bg = dict_B[key]
    n, k = Bg.shape

    if key == 'cpu':
        amins = np.argmin(Gg, axis=0)
        amins = np.array(amins).reshape(-1)

        # # loop free implementation
        eg = csr_matrix((np.ones(len(amins)), (amins, np.arange(n))), shape=A.shape)

    elif key == 'gpu_dense':
        # get argmins
        amins = cp.argmin(Gg, axis=0)

        # loop free implementation
        eg = cp.zeros((k, n))
        eg[amins, cp.arange(n)] = 1.0

    elif key == 'gpu_sparse':
        row_indices = cp.array(amins)
        col_indices = cp.arange(n)
        data = cp.array([1.0] * len(row_indices))
        eg = cupyx.scipy.sparse.coo_matrix((data, (row_indices, col_indices)), shape=(k, n))
        eg = eg.tocsr()

    egs[key] = eg


check_results(egs)

Sparsity of CPU is 0.07692307692307693
Sparsity of gpu_sparse is 0.07692307692307693
Sparsity of gpu_dense is 0.07692307692307693


In [102]:
print(len(data))

1000


In [106]:
updatedA = {} 

for key in ['cpu', 'gpu_dense', 'gpu_sparse']: 
    t = 1
    e = egs[key]
    A = dict_A[key]
    if key == 'cpu':
        updated_A = A+  2.0 / (t + 2.0) * (e - A)
    elif key == 'gpu_dense': 
        f = 2.0 / (t + 2.0)
        updated_A = cp.add(A, cp.multiply(f, cp.subtract(e, A)))
    elif key == 'gpu_sparse': 
        f = 2.0 / (t + 2.0)
        updated_A = A + (f * (e - A))
    
    updatedA[key] = updated_A

check_results(updatedA)

Sparsity of CPU is 0.9607692307692308
Sparsity of gpu_sparse is 0.9607692307692308
Sparsity of gpu_dense is 0.9607692307692308
